In [1]:
! wget -nc -O 45.zip https://osf.io/fqm4x/download

File ‘45.zip’ already there; not retrieving.


In [2]:
import zipfile
import os

In [3]:
datafolder = 'data'
os.makedirs(datafolder, exist_ok=True)

In [4]:
with zipfile.ZipFile('45.zip', 'r') as zip_ref:
    zip_ref.extractall(datafolder)

In [5]:
import pandas

In [6]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [7]:
import glob

In [8]:
Data['Folder'] = glob.glob(os.path.join('data', '*' + os.path.sep))

In [9]:
Data

,Folder
0,data/45/


In [10]:
# We could do it in a list comprehension, but then it fails if we're still scanning a tooth
# Data['LogFile'] = [sorted(glob.glob(os.path.join(f, '*.log')))[0] for f in Data['Folder']]
for c, row in Data.iterrows():
    try:
        Data.at[c, 'LogFile'] = sorted(glob.glob(os.path.join(row['Folder'], '*.log')))[0]
    except IndexError:
        print('No logfile found in %s, removing the folder temporarily' % row.Folder)
        Data.at[c, 'LogFile'] = 'scanning'
Data = Data[Data['LogFile'] != 'scanning']
Data.reset_index(drop=True, inplace=True)
print('We have %s tooth folders to work with' % (len(Data)))

We have 1 tooth folders to work with


In [11]:
Data['Sample'] = [os.path.splitext(os.path.basename(l))[0] for l in Data['LogFile']]

In [12]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f,
                                                         'rec',
                                                         '*rec*.png'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [13]:
def get_pixelsize(logfile):
    """Get the pixel size from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [14]:
Data['Voxelsize'] = [get_pixelsize(log) for log in Data['LogFile']]

In [15]:
# Check voxel sizes (*rounded* to two after-comma values)
# If different, spit out which values
if len(Data['Voxelsize'].round(2).unique()) > 1:
    print('We scanned the teeht with %s different voxel sizes' % len(Data['Voxelsize'].round(2).unique()))
    for vs in sorted(Data['Voxelsize'].round(2).unique()):
        print('-', vs, 'um for Samples ', end='')
        for c, row in Data.iterrows():
            if float(vs) == round(row['Voxelsize'], 2):
                print(row.Sample, end=', ')
        print('')
else:
    print('We scanned all datasets with equal voxel size, namely %s.' % Data['Voxelsize'].unique()[0])

We scanned all datasets with equal voxel size, namely 9.999986.
